In [1]:
import pandas as pd
import numpy as np
import requests
import string
from bs4 import BeautifulSoup

In [206]:
def scrapePage(url):
    fighters = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    rows = soup.find('table').find('tbody').find_all('tr')
    for row in rows[1:]:
        #iterate through each fighter
        attrs= row.find_all('td')
        wins = int(attrs[7].text.strip())
        losses =int(attrs[8].text.strip())
        draws = int(attrs[9].text.strip())
        if wins+losses+draws>=1:
            play_dict = {}
            play_dict['wins'] = wins
            play_dict['losses'] = losses
            play_dict['draws'] = draws
            play_dict['first_name'] = attrs[0].text.strip()
            play_dict['last_name'] = attrs[1].text.strip()
            play_dict['height'] = attrs[3].text.strip()
            play_dict['weight'] = attrs[4].text.strip()
            play_dict['reach'] = attrs[5].text.strip()
            play_dict['stance'] = attrs[6].text.strip()
            splitter = '>' + play_dict['first_name']
            
            play_dict['url'] = str(row.find('td')).split(splitter)[0].split('s/')[-1]
            fighters.append(play_dict)
        else:
            pass
    return fighters
    

In [207]:
def scrapeAll():
    letters = list(string.ascii_lowercase)
    for i, letter in enumerate(letters):
        print letter
        url = 'http://www.fightmetric.com/statistics/fighters?char={}&page=all'.format(letter)
        if i ==0:
            fighters = scrapePage(url)
        else:
            fighters+=scrapePage(url)
    df = pd.DataFrame(fighters)
    df['fighter_id'] = range(df.shape[0])
    return df

In [208]:
df= scrapeAll()

a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z


In [254]:
df = pd.read_csv('fighters_over15.csv')

In [212]:
df.head()

,draws,first_name,height,last_name,losses,reach,stance,url,weight,wins,fighter_id
0,0,David,"6' 0""",Abbott,14,--,Switch,"b361180739bed4b0""",265 lbs.,10,0
1,0,Shamil,"6' 3""",Abdurakhimov,4,"76.0""",Orthodox,"2f5cbecbbe18bac4""",235 lbs.,18,1
2,3,Hiroyuki,"5' 6""",Abe,14,--,Orthodox,"c0ed7b208197e8de""",145 lbs.,8,2
3,0,Daniel,"5' 8""",Acacio,12,--,Orthodox,"1c5879330d42255f""",180 lbs.,26,3
4,2,Sam,"6' 3""",Adkins,20,--,Orthodox,"0e9869d712e81f8f""",225 lbs.,7,4


In [ ]:
def scrapeFighterStats(fighter_url):
    #grab career statistics for each fighter
    url = 'http://www.fightmetric.com/fighter-details/{}'.format(fighter_url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

In [216]:
fighter_url = 'b361180739bed4b0'
url = 'http://www.fightmetric.com/fighter-details/{}'.format(fighter_url)
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

In [222]:
left = soup.find('ul', {'class':'b-list__box-list b-list__box-list_margin-top'}).find_all('li')

In [264]:
def careerStats(fighter_url):
    url = 'http://www.fightmetric.com/fighter-details/{}'.format(fighter_url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    dict_fighter = {}
    left = soup.find('ul', {'class':'b-list__box-list b-list__box-list_margin-top'}).find_all('li')
    right = soup.find_all('ul', {'class':'b-list__box-list b-list__box-list_margin-top'})[1].find_all('li')
    dict_fighter['url'] = fighter_url
    for row in left:
        attrs = row.text.strip()
        try:
            dict_fighter[attrs.split(':')[0]]= attrs.split(':')[1].split('n')[-1].strip()
        except:
            pass
    for row in right:
        attrs = row.text.strip()
        try:
            dict_fighter[attrs.split(':')[0]]= attrs.split(':')[1].split('n')[-1].strip()
        except:
            pass
    return dict_fighter

In [271]:
def addCareerStats(df):
    urls = df.url
    new_df = []
    for i,url in enumerate(urls):
        print i
        new_df.append(careerStats(url))
    return pd.DataFrame(new_df)

In [2]:
df = pd.read_csv('fighters_over1.csv')

In [3]:
df.shape

(3022, 11)

In [4]:
df = pd.read_csv('complete_fighters.csv')

In [5]:
df.head()

,draws,first_name,height,last_name,losses,reach,stance,url,weight,wins,fighter_id,SApM,SLpM,Str. Acc.,Str. Def,Sub. Avg.,TD Acc.,TD Avg.,TD Def.
0,0,Tom,--,Aaron,3,--,NaN,93fe7332d16c6ad9,155 lbs.,5,0,0.00,0.00,0%,0%,0.0,0%,0.00,0%
1,0,Danny,"5' 11""",Abbadi,6,--,Orthodox,15df64c02b6b0fde,155 lbs.,4,1,4.41,3.29,38%,57%,0.0,0%,0.00,77%
2,0,David,"6' 0""",Abbott,14,--,Switch,b361180739bed4b0,265 lbs.,10,2,3.55,1.35,30%,38%,0.0,33%,1.07,66%
3,0,Shamil,"6' 3""",Abdurakhimov,4,"76.0""",Orthodox,2f5cbecbbe18bac4,235 lbs.,18,3,2.53,2.53,45%,58%,0.3,22%,1.36,77%
4,3,Hiroyuki,"5' 6""",Abe,14,--,Orthodox,c0ed7b208197e8de,145 lbs.,8,4,3.11,1.71,36%,63%,0.0,0%,0.00,33%


In [ ]:
#for each fighter, add a row for all their unique fights
#make sure if a fight is already there that you don't add again
fights = []
